## Loads

In [1]:
import sys
import os
script_dir = os.getcwd()
root_dir = os.path.join(os.path.dirname(os.path.abspath(script_dir)))
sys.path.append(os.path.join(os.path.dirname(os.path.abspath(script_dir))))
os.chdir(root_dir)

In [2]:
os.getcwd()

'/home/helenajun/rag-llm-cancer-paper'

In [3]:
# !pip install rank_bm25
import json
import pandas as pd
import numpy as np
from utils.io import load_object, save_object
import requests
import random
import math
from transformers import AutoTokenizer, AutoModelForTokenClassification
from utils.entity_prediction import extract_entities, db_extract_entities
import ast

In [4]:
#local db version
CACHE_FILE = "db_version_cache.json"

def get_remote_version():
    agents = requests.get('https://api.moalmanac.org/agents').json()
    return agents['service']['last_updated']

def get_local_version():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE) as f:
            return json.load(f).get("version")
    return None

def save_local_version(version):
    with open(CACHE_FILE, "w") as f:
        json.dump({"version": version}, f)


#split ranges for testing on small batches
def split_ranges(max_int, num_ranges, samples=None, seed=42):
    random.seed(seed)
    step = math.ceil(max_int / num_ranges)
    ranges = [(i*step+1, min((i+1)*step, max_int)) for i in range(num_ranges)]
    
    if samples is None:
        return ranges
    
    # split samples evenly across ranges
    n_per_range = samples // num_ranges
    remainder = samples % num_ranges
    
    result = []
    for i, r in enumerate(ranges):
        count = n_per_range + (1 if i < remainder else 0)
        result.extend(random.randint(r[0], r[1]) for _ in range(count))
    
    return result

#get ground-truth drug entry from db for evaluation
def get_true_drug_db(moalmanac_db, db_idx):
    subset = moalmanac_db[
        (moalmanac_db["modified_standardized_cancer"] == moalmanac_db["modified_standardized_cancer"].iloc[db_idx]) &
        (moalmanac_db["biomarker"] == moalmanac_db["biomarker"].iloc[db_idx])
    ]
    return subset

_VERSION=get_local_version()
_VERSION

'2025-09-04'

## Test hybrid search for RAG

BM25 (sparse retrieval): 
- works well when exact terms in query match documents, so it's good for keyword-heavy queries
- considers document length, so longer documents don't get unfairly higher scores because they contain more words, which is the problem in naive entity matching 

FAISS cosine similarity (dense retrieval): 
- captures semantic similarity (even if exact words don't match) 

Hybrid approach:
1. Use BM25 to retrieve a short list of relevant documents
2. Retrieve top-k based on vector similarity using FAISS
3. Re-rank with a cross-encoder

In [5]:
#test hybrid matching
from typing import List, Dict
import faiss
from utils.embedding import get_text_embedding, prep_embed_for_search, retrieve_context
from utils.hybrid_search import retrieve_context_hybrid
from dotenv import load_dotenv
from openai import OpenAI
from rank_bm25 import BM25Okapi


def check_list(input) -> List[str]:
    if isinstance(input, list):
        input = input
    else:
        input = [input]
    return input


def tokenize_single_entity(entity: str) -> List[str]:
    tokens = entity.split()
    return tokens


def tokenize_corpus(
    corpus: list[dict[str, list[str]]], 
    entity=['cancer_type', 'biomarker']
    ) -> Dict[List[str], List[str]]:
    tokenized_corpus = []
    for doc in corpus:
        tokenized_doc = {}
        for ent in entity:
            tokenized_ent = []
            for e in doc[ent]:
                if e is not None and isinstance(e, str):
                    tokenized_ent.extend(tokenize_single_entity(e))
                    
            tokenized_doc[ent] = tokenized_ent
            
        tokenized_corpus.append(tokenized_doc)
    
    return tokenized_corpus


def min_max_scaling(score: np.ndarray, eps=1e-8) -> np.ndarray:
    norm_score=(score - score.min())/(score.max()-score.min()+eps)
    return norm_score
    

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
_CLIENT = OpenAI(api_key=api_key)
_MODEL_EMBED = "text-embedding-3-small"
_INDEX=faiss.read_index(f"./data/latest_db/indexes/text-embedding-3-small_structured_context__{_VERSION}.faiss")
with open(f"context_retriever/entities/moalmanac_db_ner_entities__{_VERSION}.json", "r") as f:
    _DB_ENTITY = json.load(f)
with open(f"context_retriever/entities/synthetic_query_ner_entities__{_VERSION}.json", "r") as f:
    _QUERY_ENTITY = json.load(f)
with open(f"data/latest_db/indexes/text-embedding-3-small_structured_context__{_VERSION}.json", "r") as f:
    _CONTEXT = json.load(f)
moalmanac_db = pd.read_csv(f"data/latest_db/moalmanac_core__{_VERSION}.csv")
with open(f"data/latest_db/synthetic_prompts__{_VERSION}.json", "r") as f:
    synthetic_query=json.load(f)
synthetic_query_therapy_pair_dict=load_object("data/latest_db/synthetic_query_therapy_pair_dict.pkl")
small_val_idx = split_ranges(len(_CONTEXT), 10, 20)
print(small_val_idx)

[15, 4, 101, 97, 159, 148, 209, 207, 315, 265, 329, 337, 418, 420, 520, 459, 546, 574, 600, 614]


In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Any

@dataclass
class HybridRetrievalResult:
    top_contexts: List[str] # final list of top re-ranked contexts
    retrieved_df: pd.DataFrame  # detailed DataFrame with all scores
    params: Dict[str, Any]


def retrieve_context_hybrid(
    user_entities: dict[str, list[str]], 
    db_entities: list[dict[str, list[str]]],
    user_query: str,
    db_context,
    index,
    client,
    model_embed,
    bm25_top_k=25,
    w_c=0.3,
    w_b=0.3,
    w_ctx=0.4,
    num_vec=10
    ) -> HybridRetrievalResult:
    """
    Performs hybrid search and retrieval with two stages:
    1. Sparse retrieval (BM25):
        - Uses keyword-based scoring on tokenized entities (`cancer_type` and `biomarker`)
        - Combines entity-level BM25 scores with pre-defined weights
        - Selects top-k candidate contexts
    2. Dense re-ranking (FAISS):
        - Computes query embeddings with the given model
        - Retrieves semantically similar context using FAISS cosine similarity
        - Aligns FAISS scores with BM25-selected contexts and re-ranks
    
    Returns:
        - A list of top `num_vec` re-ranked contexts
        - A DataFrame containing context, BM25 scores, FAISS similarity, and final rankings score
    
    Notes:
        - BM25 captures lexical (exact keyword match) signals
        - FAISS cosine similarity captures semantic similarity, ensuring better ranking of relevant contexts with synonyms
        - Final ranking score is a product of BM25 score and cosine similarity for balanced ranking
    """
    
    #extract user cancer/biomarker entities (handles none)
    user_cancer = check_list(user_entities.get('cancer_type', []))
    user_biomarker = check_list(user_entities.get('biomarker', []))
    
    #tokenize user query and corpus
    tokenized_user_query=tokenize_single_entity(user_query)
    tokenized_db_context=[tokenize_single_entity(ctx) for ctx in db_context]
    
    #tokenize user and db entities (cancer/biomarker) for fine-grained matching
    tokenized_db_entities=tokenize_corpus(db_entities)
    tokenized_user_cancer = [token for e in user_cancer if e for token in tokenize_single_entity(e)]
    tokenized_user_biomarker = [token for e in user_biomarker if e for token in tokenize_single_entity(e)]
    
    #build BM25 indices for each type of information
    cancer_bm25=BM25Okapi([doc['cancer_type'] for doc in tokenized_db_entities])
    biomarker_bm25=BM25Okapi([doc['biomarker'] for doc in tokenized_db_entities])
    context_bm25=BM25Okapi(tokenized_db_context)
    
    #bm25 scoring for user entities (captures lexical match; one caveat is that it relies on NER quality)
    if tokenized_user_cancer:
        cancer_bm25_score=cancer_bm25.get_scores(tokenized_user_cancer)
        #normalized to [0,1]
        cancer_bm25_score_norm=min_max_scaling(cancer_bm25_score)
        #zero score vector if no entity detected 
    else: cancer_bm25_score_norm=np.zeros(len(db_context)) 
    if tokenized_user_biomarker:
        biomarker_bm25_score=biomarker_bm25.get_scores(tokenized_user_biomarker)
        biomarker_bm25_score_norm=min_max_scaling(biomarker_bm25_score)
    else: biomarker_bm25_score_norm=np.zeros(len(db_context))
    
    #bm25 scoring for free-text contexts (captures info missed by entity extraction like disease modifiers)
    context_bm25_score=context_bm25.get_scores(tokenized_user_query)
    context_bm25_score_norm=min_max_scaling(context_bm25_score)    
    
    #combine bm25 scores (weighted sum of the three normalized BM25 scores with weight highest for context)
    combined_score = w_c * cancer_bm25_score_norm + w_b * biomarker_bm25_score_norm + w_ctx * context_bm25_score_norm
    
    #fall back entirely on FAISS semantic similarity when there is low BM25 combined score
    if np.max(combined_score) < 0.1:
        retrieved_chunks, D, I = retrieve_context(
            db_context, 
            user_query,
            client,
            model_embed,
            index,
            num_vec
            )
        faiss_cosine_sim_norm_ordered=(D.ravel()+1)/2
        
        #return only semantic search results
        retrieved_combined=pd.DataFrame({
            'bm25_retrieved_context':None,
            'bm25_index':None,
            'bm25_combined_score':None,
            'faiss_retrieved_context':retrieved_chunks,
            'faiss_cosine_sim':faiss_cosine_sim_norm_ordered
            })
        
    else:
        #top BM25 results
        bm25_top_idx = np.argsort(combined_score)[-bm25_top_k:][::-1]
        bm25_top_scores = combined_score[bm25_top_idx]
        bm25_top_context = [db_context[i] for i in bm25_top_idx]
        
        #FAISS re-ranking on BM25 candidates (captures semantic similarity)
        factor = max(2, min(10, index.ntotal // bm25_top_k)) #determines num. of FAISS neighbors to retrieve
        query_embeddings=np.array([get_text_embedding(user_query, client, model_embed)])
        query_embeddings_norm = query_embeddings/np.linalg.norm(query_embeddings, axis=1, keepdims=True)
        D, I = index.search(prep_embed_for_search(query_embeddings_norm, n_dim=2), k=bm25_top_k*factor)
        faiss_cosine_sim=(D.ravel()+1)/2
        faiss_indices=I.ravel()
        faiss_sim_lookup=dict(zip(faiss_indices, faiss_cosine_sim))
        faiss_cosine_sim_norm_ordered=[faiss_sim_lookup.get(idx, 0) for idx in bm25_top_idx]
        
        #combine BM25+FAISS for final reranking
        retrieved_combined=pd.DataFrame({
            'bm25_retrieved_context':bm25_top_context,
            'bm25_index':bm25_top_idx,
            'bm25_combined_score':bm25_top_scores,
            'faiss_retrieved_context':None,
            'faiss_cosine_sim':faiss_cosine_sim_norm_ordered
            })
        #hybrid score
        retrieved_combined['final_ranking_score']=retrieved_combined['faiss_cosine_sim']*retrieved_combined['bm25_combined_score']
        retrieved_combined=retrieved_combined.sort_values('final_ranking_score', ascending=False)
        
        #top-n final context chunks
        retrieved_chunks=retrieved_combined['bm25_retrieved_context'].head(num_vec).tolist()
        
    params = {
        'bm25_top_k':bm25_top_k,
        'w_c':w_c,
        'w_b':w_b,
        'w_ctx':w_ctx,
        'factor':factor,
        'num_vec':num_vec
    }
    
    return HybridRetrievalResult(
        top_contexts=retrieved_chunks, 
        retrieved_df=retrieved_combined,
        params=params            
    )


Sparse and dense search retrieval steps:
1. Cancer type, biomarker, context search using BM20 scoring:
    - combining BM20 scores from each search with weights: cancer type, biomarker, and context

2. Dense semantic similarity search using FAISS cosine similarity
3. Combine 1 and 2 scores for final re-ranking. If no entities are found in 1., then lean to semantic search 2. only

## Quick evaluation

In [8]:
moalmanac_db_query=pd.concat([moalmanac_db, pd.Series(synthetic_query)],axis=1)
moalmanac_db_query=moalmanac_db_query.rename(columns={0:'prompt'})
for entry in moalmanac_db_query.loc[small_val_idx].itertuples():
    print(entry.Index)
    print(entry.prompt)
    print(_QUERY_ENTITY[entry.Index])

15
if a patient with metastatic castration-resistant prostate adenocarcinoma cancer has bard1 oncogenic variants, what therapy is recommended?
{'cancer_type': ['prostate adenocarcinoma cancer'], 'biomarker': ['bard1']}
4
if a patient with advanced or metastatic invasive breast carcinoma cancer has pr positive, her2-negative, what therapy is recommended?
{'cancer_type': ['invasive breast carcinoma cancer'], 'biomarker': ['her2', 'pr']}
101
if a patient with unresectable or metastatic any solid tumor cancer has braf p.v600e, what therapy is recommended?
{'cancer_type': ['solid tumor cancer'], 'biomarker': []}
97
if a patient with melanoma cancer has braf p.v600k, what therapy is recommended?
{'cancer_type': ['melanoma cancer'], 'biomarker': []}
159
if a patient with advanced or metastatic invasive breast carcinoma cancer has pr positive, her2-negative, er positive, what therapy is recommended?
{'cancer_type': ['invasive breast carcinoma cancer'], 'biomarker': ['er', 'her2', 'pr']}
148
if

### Test which alpha to use for final combined score: alpha*bm25_combined_score + (1-alpha)*faiss_cosine_sim

In [10]:
#function to calculate recall at top_k retrieved context
def recall_at_k(moalmanac_db, predicted_chunks_idx, ground_truth_drugs, k=10):
    """
    predicted_chunks: list of retrieved context idx, ranked by BM25 score
    ground_truth_drugs: set of drug names considered correct for this query
    k: top-k cutoff
    """
    retrieved_drugs = [drug for drug in moalmanac_db.loc[predicted_chunks_idx].therapy.apply(ast.literal_eval)][:k]
    retrieved_drugs = [{d.lower() for d in d_set} for d_set in retrieved_drugs]

    ground_truth_drugs = frozenset(frozenset(d_true) for d_true in ground_truth_drugs)
    retrieved_drugs_unique = frozenset(frozenset(d_set) for d_set in retrieved_drugs if d_set in ground_truth_drugs)

    acc = len(retrieved_drugs_unique & ground_truth_drugs)/len(ground_truth_drugs)
    
    return acc

Test on small batch of samples

In [34]:
val_idx=small_val_idx
val_queries=np.array(synthetic_query)[val_idx]
alphas = np.linspace(0, 1, 6)
k = 10

best_alpha = None
best_recall = -1

for alpha in alphas:
    recalls = []
    for idx, q in zip(val_idx, np.array(synthetic_query)[val_idx]):
        #get candidate context indices and their BM25 and FAISS scores
        retrieved_res = retrieve_context_hybrid(
            user_entities=_QUERY_ENTITY[idx], 
            db_entities=_DB_ENTITY,
            user_query=q,
            db_context=_CONTEXT,
            index=_INDEX,
            client=_CLIENT,
            model_embed=_MODEL_EMBED
        )
        
        #retrieved context sorted by bm25 scores
        retrieved_res = retrieved_res.retrieved_df.sort_values('bm25_combined_score', ascending=False)
        candidates = retrieved_res.bm25_index
        bm25_scores = retrieved_res.bm25_combined_score
        faiss_scores = retrieved_res.faiss_cosine_sim
        
        #final score
        final_score = alpha*bm25_scores + (1-alpha)*faiss_scores
        
        #rank candidates based on final score
        ranked_candidates = [c for _, c in sorted(zip(final_score, candidates), reverse=True)]
        
        # compute recall@k
        r = recall_at_k(moalmanac_db, ranked_candidates, synthetic_query_therapy_pair_dict[q], k=k)
        recalls.append(r)
    
    mean_recall = np.mean(recalls)
    print(f"alpha={alpha:.1f} -> mean recall@{k}={mean_recall:.3f}")
    
    if mean_recall > best_recall:
        best_recall = mean_recall
        best_alpha = alpha

print(f"Best alpha: {best_alpha}, recall@{k}={best_recall:.3f}")


alpha=0.0 -> mean recall@10=0.912
alpha=0.2 -> mean recall@10=0.972
alpha=0.4 -> mean recall@10=0.962
alpha=0.6 -> mean recall@10=0.932
alpha=0.8 -> mean recall@10=0.921
alpha=1.0 -> mean recall@10=0.921
Best alpha: 0.2, recall@10=0.972


Test on a larger batch of samples

In [36]:
#balanced sampling of validation set for grid search
from sklearn.model_selection import train_test_split

val_df = moalmanac_db.groupby('modified_standardized_cancer').apply(
    lambda x: x.sample(min(len(x), 3), random_state=42)
)

val_idx=np.array(moalmanac_db[moalmanac_db['statement_id'].isin(val_df.statement_id)].index)
print(len(val_idx))

237


In [37]:
val_queries=np.array(synthetic_query)[val_idx]
alphas = np.linspace(0, 1, 6)
k = 10

best_alpha = None
best_recall = -1

for alpha in alphas:
    recalls = []
    for idx, q in zip(val_idx, np.array(synthetic_query)[val_idx]):
        #get candidate context indices and their BM25 and FAISS scores
        retrieved_res = retrieve_context_hybrid(
            user_entities=_QUERY_ENTITY[idx], 
            db_entities=_DB_ENTITY,
            user_query=q,
            db_context=_CONTEXT,
            index=_INDEX,
            client=_CLIENT,
            model_embed=_MODEL_EMBED
        )
        
        #sort by bm25 scores
        retrieved_res = retrieved_res.retrieved_df.sort_values('bm25_combined_score', ascending=False)
        candidates = retrieved_res.bm25_index
        bm25_scores = retrieved_res.bm25_combined_score
        faiss_scores = retrieved_res.faiss_cosine_sim
        
        #final score
        final_score = alpha*bm25_scores + (1-alpha)*faiss_scores
        
        #rank candidates based on final score
        ranked_candidates = [c for _, c in sorted(zip(final_score, candidates), reverse=True)]
        
        # compute recall@k
        r = recall_at_k(moalmanac_db, ranked_candidates, synthetic_query_therapy_pair_dict[q], k=k)
        recalls.append(r)
    
    mean_recall = np.mean(recalls)
    print(f"alpha={alpha:.1f} -> mean recall@{k}={mean_recall:.3f}")
    
    if mean_recall > best_recall:
        best_recall = mean_recall
        best_alpha = alpha

print(f"Best alpha: {best_alpha}, recall@{k}={best_recall:.3f}")


alpha=0.0 -> mean recall@10=0.930
alpha=0.2 -> mean recall@10=0.991
alpha=0.4 -> mean recall@10=0.990
alpha=0.6 -> mean recall@10=0.988
alpha=0.8 -> mean recall@10=0.989
alpha=1.0 -> mean recall@10=0.989
Best alpha: 0.2, recall@10=0.991


### Inspect which queries benefit the most from alpha tuning

In [78]:
alpha_0 = 0 #pure faiss
alpha_best = 0.2 
val_idx=small_val_idx

retrieved_res_ls=[]
recall_0=[]
recall_best=[]
r_diff=[]
for idx, q in zip(val_idx, np.array(synthetic_query)[val_idx]):
    retrieved_res = retrieve_context_hybrid(
            user_entities=_QUERY_ENTITY[idx], 
            db_entities=_DB_ENTITY,
            user_query=q,
            db_context=_CONTEXT,
            index=_INDEX,
            client=_CLIENT,
            model_embed=_MODEL_EMBED
        )
    retrieved_res = retrieved_res.retrieved_df.sort_values('bm25_combined_score', ascending=False)
    candidates = retrieved_res.bm25_index
    bm25_scores = retrieved_res.bm25_combined_score
    faiss_scores = retrieved_res.faiss_cosine_sim
    retrieved_res_ls.append(retrieved_res)
    
    hybrid_0 = alpha_0*bm25_scores + (1-alpha_0)*faiss_scores
    hybrid_best = alpha_best*bm25_scores + (1-alpha_best)*faiss_scores
    
    ranked_candidates_0 = [c for _, c in sorted(zip(hybrid_0, candidates), reverse=True)]
    ranked_candidates_best = [c for _, c in sorted(zip(hybrid_best, candidates), reverse=True)]
    
    r_0 = recall_at_k(moalmanac_db, ranked_candidates_0, synthetic_query_therapy_pair_dict[q], k=k)
    r_best = recall_at_k(moalmanac_db, ranked_candidates_best, synthetic_query_therapy_pair_dict[q], k=k)
    recall_0.append(r_0)
    recall_best.append(r_best)
    r_diff.append((r_best-r_0)/r_0)

In [90]:
df_gain = pd.DataFrame({
    'query_idx': small_val_idx,
    'query': np.array(synthetic_query)[small_val_idx],
    'recall_alpha_0': recall_0,
    'recall_alpha_best': recall_best,
    'recall_gain': r_diff
    })

df_gain.sort_values("recall_gain", ascending=False)

,query_idx,query,recall_alpha_0,recall_alpha_best,recall_gain
7,207,if a patient with acute myeloid leukemia cance...,0.500000,1.000000,1.000000
5,148,if a patient with invasive breast carcinoma ca...,0.666667,1.000000,0.500000
19,614,if a patient with early invasive breast carcin...,0.833333,1.000000,0.200000
13,420,if a patient with early invasive breast carcin...,0.900000,1.000000,0.111111
12,418,if a patient with early invasive breast carcin...,0.900000,1.000000,0.111111
11,337,if a patient with metastatic castration-resist...,1.000000,1.000000,0.000000
18,600,if a patient with pancreatic adenocarcinoma ca...,1.000000,1.000000,0.000000
17,574,if a patient with oligodendroglioma cancer has...,1.000000,1.000000,0.000000
16,546,if a patient with high-grade serous fallopian ...,1.000000,1.000000,0.000000
15,459,if a patient with high-grade serous fallopian ...,1.000000,1.000000,0.000000


In [107]:
recall_gain_idx=df_gain.sort_values("recall_gain", ascending=False).head(5).query_idx.values
recall_gain_df_idx=df_gain.sort_values("recall_gain", ascending=False).head(5).index

In [117]:
for idx, idx_df in zip(recall_gain_idx, recall_gain_df_idx):
    print(f"#============={idx}=============#")
    print(synthetic_query[idx])
    print(_QUERY_ENTITY[idx])
    print(_CONTEXT[idx])
    print(retrieved_res_ls[idx_df].head(5))
    print(get_true_drug_db(moalmanac_db, db_idx=idx).index)
    print(synthetic_query_therapy_pair_dict[synthetic_query[idx]])

#=============207=============#
if a patient with acute myeloid leukemia cancer has idh1 p.r132l, what therapy is recommended?
{'cancer_type': ['acute myeloid leukemia cancer'], 'biomarker': ['idh1']}
If a patient with acute myeloid leukemia cancer has idh1 p.r132l, one recommended therapy is ivosidenib + azacitidine. therapy type: targeted therapy + chemotherapy. therapy strategy: idh1 inhibition + hypomethylating agent chemotherapy. indication: tibsovo is an isocitrate dehydrogenase-1 (idh1) inhibitor indicated for patients with a susceptible idh1 mutation as detected by an fda-approved test with in combination with azacitidine or as monotherapy for the treatment of newly diagnosed aml in adults 75 years or older, or who have comorbidities that preclude use of intensive induction chemotherapy. approval url: https://www.accessdata.fda.gov/drugsatfda_docs/label/2023/211192s011lbl.pdf
                              bm25_retrieved_context  bm25_index  \
0  If a patient with relapsed or re

### Final retrieval pipeline with the best alpha

In [118]:
from dataclasses import dataclass
from typing import List, Dict, Any

@dataclass
class HybridRetrievalResult:
    top_contexts: List[str] # final list of top re-ranked contexts
    retrieved_df: pd.DataFrame  # detailed DataFrame with all scores
    params: Dict[str, Any]


def retrieve_context_hybrid(
    user_entities: dict[str, list[str]], 
    db_entities: list[dict[str, list[str]]],
    user_query: str,
    db_context,
    index,
    client,
    model_embed,
    bm25_top_k=25,
    w_c=0.3,
    w_b=0.3,
    w_ctx=0.4,
    alpha=0.2,
    num_vec=10
    ) -> HybridRetrievalResult:
    """
    Performs hybrid search and retrieval with two stages:
    1. Sparse retrieval (BM25):
        - Uses keyword-based scoring on tokenized entities (`cancer_type` and `biomarker`)
        - Combines entity-level BM25 scores with pre-defined weights
        - Selects top-k candidate contexts
    2. Dense re-ranking (FAISS):
        - Computes query embeddings with the given model
        - Retrieves semantically similar context using FAISS cosine similarity
        - Aligns FAISS scores with BM25-selected contexts and re-ranks
    
    Returns:
        - A list of top `num_vec` re-ranked contexts
        - A DataFrame containing context, BM25 scores, FAISS similarity, and final rankings score
    
    Notes:
        - BM25 captures lexical (exact keyword match) signals
        - FAISS cosine similarity captures semantic similarity, ensuring better ranking of relevant contexts with synonyms
        - Final ranking score is a product of BM25 score and cosine similarity for balanced ranking
    """
    
    #extract user cancer/biomarker entities (handles none)
    user_cancer = check_list(user_entities.get('cancer_type', []))
    user_biomarker = check_list(user_entities.get('biomarker', []))
    
    #tokenize user query and corpus
    tokenized_user_query=tokenize_single_entity(user_query)
    tokenized_db_context=[tokenize_single_entity(ctx) for ctx in db_context]
    
    #tokenize user and db entities (cancer/biomarker) for fine-grained matching
    tokenized_db_entities=tokenize_corpus(db_entities)
    tokenized_user_cancer = [token for e in user_cancer if e for token in tokenize_single_entity(e)]
    tokenized_user_biomarker = [token for e in user_biomarker if e for token in tokenize_single_entity(e)]
    
    #build BM25 indices for each type of information
    cancer_bm25=BM25Okapi([doc['cancer_type'] for doc in tokenized_db_entities])
    biomarker_bm25=BM25Okapi([doc['biomarker'] for doc in tokenized_db_entities])
    context_bm25=BM25Okapi(tokenized_db_context)
    
    #bm25 scoring for user entities (captures lexical match; one caveat is that it relies on NER quality)
    if tokenized_user_cancer:
        cancer_bm25_score=cancer_bm25.get_scores(tokenized_user_cancer)
        #normalized to [0,1]
        cancer_bm25_score_norm=min_max_scaling(cancer_bm25_score)
        #zero score vector if no entity detected 
    else: cancer_bm25_score_norm=np.zeros(len(db_context)) 
    if tokenized_user_biomarker:
        biomarker_bm25_score=biomarker_bm25.get_scores(tokenized_user_biomarker)
        biomarker_bm25_score_norm=min_max_scaling(biomarker_bm25_score)
    else: biomarker_bm25_score_norm=np.zeros(len(db_context))
    
    #bm25 scoring for free-text contexts (captures info missed by entity extraction like disease modifiers)
    context_bm25_score=context_bm25.get_scores(tokenized_user_query)
    context_bm25_score_norm=min_max_scaling(context_bm25_score)    
    
    #combine bm25 scores (weighted sum of the three normalized BM25 scores with weight highest for context)
    combined_score = w_c * cancer_bm25_score_norm + w_b * biomarker_bm25_score_norm + w_ctx * context_bm25_score_norm
    
    #fall back entirely on FAISS semantic similarity when there is low BM25 combined score
    if np.max(combined_score) < 0.1:
        retrieved_chunks, D, I = retrieve_context(
            db_context, 
            user_query,
            client,
            model_embed,
            index,
            num_vec
            )
        faiss_cosine_sim_norm_ordered=(D.ravel()+1)/2
        
        #return only semantic search results
        retrieved_combined=pd.DataFrame({
            'bm25_retrieved_context':None,
            'bm25_index':None,
            'bm25_combined_score':None,
            'faiss_retrieved_context':retrieved_chunks,
            'faiss_cosine_sim':faiss_cosine_sim_norm_ordered
            })
        
    else:
        #top BM25 results
        bm25_top_idx = np.argsort(combined_score)[-bm25_top_k:][::-1]
        bm25_top_scores = combined_score[bm25_top_idx]
        bm25_top_context = [db_context[i] for i in bm25_top_idx]
        
        #FAISS re-ranking on BM25 candidates (captures semantic similarity)
        factor = max(2, min(10, index.ntotal // bm25_top_k)) #determines num. of FAISS neighbors to retrieve
        query_embeddings=np.array([get_text_embedding(user_query, client, model_embed)])
        query_embeddings_norm = query_embeddings/np.linalg.norm(query_embeddings, axis=1, keepdims=True)
        D, I = index.search(prep_embed_for_search(query_embeddings_norm, n_dim=2), k=bm25_top_k*factor)
        faiss_cosine_sim=(D.ravel()+1)/2
        faiss_indices=I.ravel()
        faiss_sim_lookup=dict(zip(faiss_indices, faiss_cosine_sim))
        faiss_cosine_sim_norm_ordered=[faiss_sim_lookup.get(idx, 0) for idx in bm25_top_idx]
        
        #combine BM25+FAISS for final reranking
        retrieved_combined=pd.DataFrame({
            'bm25_retrieved_context':bm25_top_context,
            'bm25_index':bm25_top_idx,
            'bm25_combined_score':bm25_top_scores,
            'faiss_retrieved_context':None,
            'faiss_cosine_sim':faiss_cosine_sim_norm_ordered
            })
        
        #hybrid score
        retrieved_combined['final_ranking_score'] = alpha * retrieved_combined['bm25_combined_score'] + (1-alpha) * retrieved_combined['faiss_cosine_sim']
        retrieved_combined=retrieved_combined.sort_values('final_ranking_score', ascending=False)
        
        #top-n final context chunks
        retrieved_chunks=retrieved_combined['bm25_retrieved_context'].head(num_vec).tolist()
        
    params = {
        'bm25_top_k':bm25_top_k,
        'w_c':w_c,
        'w_b':w_b,
        'w_ctx':w_ctx,
        'factor':factor,
        'alpha':alpha,
        'num_vec':num_vec
    }
    
    return HybridRetrievalResult(
        top_contexts=retrieved_chunks, 
        retrieved_df=retrieved_combined,
        params=params            
    )


In [120]:
val_idx=small_val_idx
recalls = []
for idx, q in zip(val_idx, np.array(synthetic_query)[val_idx]):
    #get candidate context indices and their BM25 and FAISS scores
    retrieved_res = retrieve_context_hybrid(
        user_entities=_QUERY_ENTITY[idx], 
        db_entities=_DB_ENTITY,
        user_query=q,
        db_context=_CONTEXT,
        index=_INDEX,
        client=_CLIENT,
        model_embed=_MODEL_EMBED
    )
    
    #retrieved context sorted by final scores
    candidates = retrieved_res.retrieved_df.bm25_index
    
    # compute recall@k
    r = recall_at_k(moalmanac_db, candidates, synthetic_query_therapy_pair_dict[q], k=k)
    recalls.append(r)

mean_recall = np.mean(recalls)
print(mean_recall)

0.9722222222222221


### Testing the pipeline with updated retrieval process

In [12]:
from utils.evaluation import calc_eval_metrics
synthetic_query_therapy_pair_dict=load_object("data/latest_db/synthetic_query_therapy_pair_dict.pkl")

Update all generic names set for evaluation

In [ ]:
_SAVE=False
if _SAVE:
    all_generic_names_set={frozenset(_set) for k, v in synthetic_query_therapy_pair_dict.items() for _set in v}
    save_object(all_generic_names_set, "data/latest_db/all_generic_drugs.pkl")

#### Test on a small subset of samples (n=20)

In [13]:
moalmanac_db = pd.read_csv(f"data/latest_db/moalmanac_core__{_VERSION}.csv")
moalmanac_db_query = pd.concat([moalmanac_db, pd.Series(synthetic_query)], axis=1)
moalmanac_db_query=moalmanac_db_query.rename(columns={0:'prompt'})
moalmanac_db_query.loc[small_val_idx].to_csv("data/latest_db/moalmanac_small_testing_dataset__v2.csv", index=True)

In [8]:
moalmanac_db_query_subset=pd.read_csv("data/latest_db/moalmanac_small_testing_dataset__v2.csv", index_col=0)
moalmanac_db_query_subset

,statement_id,standardized_cancer,raw_cancer,modified_standardized_cancer,biomarker,therapy,prompt
15,15,Prostate Adenocarcinoma,metastatic castration-resistant prostate cancer,metastatic castration-resistant prostate adeno...,['BARD1 oncogenic variants'],"['Prednisone', 'Niraparib', 'Abiraterone aceta...",if a patient with metastatic castration-resist...
4,4,Invasive Breast Carcinoma,advanced or metastatic breast cancer,advanced or metastatic invasive breast carcinoma,"['PR positive', 'HER2-negative']","['Anastrozole', 'Abemaciclib']",if a patient with advanced or metastatic invas...
101,102,Any solid tumor,unresectable or metastatic solid tumors,unresectable or metastatic any solid tumor,['BRAF p.V600E'],"['Trametinib', 'Dabrafenib']",if a patient with unresectable or metastatic a...
97,98,Melanoma,melanoma,melanoma,['BRAF p.V600K'],"['Trametinib', 'Dabrafenib']",if a patient with melanoma cancer has braf p.v...
159,160,Invasive Breast Carcinoma,advanced or metastatic breast cancer,advanced or metastatic invasive breast carcinoma,"['PR positive', 'HER2-negative', 'ER positive']","['Fulvestrant', 'Ribociclib']",if a patient with advanced or metastatic invas...
148,149,Invasive Breast Carcinoma,breast cancer,invasive breast carcinoma,"['HER2-negative', 'ER positive']","['Exemestane', 'Everolimus']",if a patient with invasive breast carcinoma ca...
209,210,Acute Myeloid Leukemia,relapsed or refractory AML,relapsed or refractory acute myeloid leukemia,['IDH1 p.R132C'],['Ivosidenib'],if a patient with relapsed or refractory acute...
207,208,Acute Myeloid Leukemia,acute myeloid leukemia,acute myeloid leukemia,['IDH1 p.R132L'],"['Ivosidenib', 'Azacitidine']",if a patient with acute myeloid leukemia cance...
315,316,Prostate Adenocarcinoma,metastatic castration-resistant prostate cancer,metastatic castration-resistant prostate adeno...,['BARD1 oncogenic variants'],['Olaparib'],if a patient with metastatic castration-resist...
265,266,Ovarian Epithelial Tumor,"advanced epithelial ovarian, fallopian tube or...",advanced ovarian epithelial tumor,['BRCA2 pathogenic variants'],['Olaparib'],if a patient with advanced ovarian epithelial ...


With dense search only

In [30]:
dense_output=load_object("output/RAG_res_gpt4o/structured_latest_db_prompts_humantextctxv1/RAGbatchstra0n1temp0.0_res_dict.pkl")
print(f"{dense_output['runtime'][0]/60:.2f} min")
dense_output_eval = calc_eval_metrics(dense_output['full output'][0], moalmanac_db_query_subset['prompt'].tolist(), synthetic_query_therapy_pair_dict)
dense_output_eval

2.24 min


{'avg_exact_match_acc': 0.4,
 'avg_partial_match_acc': 0.95,
 'avg_precision': 0.8722222222222223,
 'avg_recall': 0.6794444444444444,
 'avg_f1': 0.7065590111642744,
 'avg_specificity': 0.9982944462701282,
 'exact_match_acc': [False,
  False,
  True,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  False],
 'partial_match_acc': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False],
 'precision_ls': [1.0,
  1.0,
  1.0,
  0.3333333333333333,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.8888888888888888,
  0.8888888888888888,
  1.0,
  0.3333333333333333,
  1.0,
  1.0,
  1.0,
  0],
 'recall_ls': [0.5,
  0.2222222222222222,
  1.0,
  1.0,
  0.3333333333333333,
  0.3333333333333333,
  0.5,
  0.5,
  0.5,
  0.5,
  1.0,
  0.6,
  0.8,
  0.8,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0],
 'f1_l

In [31]:
dense_output=load_object("output/RAG_res_gpt4o/structured_latest_db_prompts_humantextctxv1/RAGstra0n1temp0.0_res_dict.pkl")
print(f"{dense_output['runtime'][0]/60:.2f} min")
dense_output_eval = calc_eval_metrics(dense_output['full output'][0], moalmanac_db_query_subset['prompt'].tolist(), synthetic_query_therapy_pair_dict)
dense_output_eval

0.70 min


{'avg_exact_match_acc': 0.4,
 'avg_partial_match_acc': 0.95,
 'avg_precision': 0.8722222222222223,
 'avg_recall': 0.6794444444444444,
 'avg_f1': 0.7065590111642744,
 'avg_specificity': 0.9980087319844138,
 'exact_match_acc': [False,
  False,
  True,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  False],
 'partial_match_acc': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False],
 'precision_ls': [1.0,
  1.0,
  1.0,
  0.3333333333333333,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.8888888888888888,
  0.8888888888888888,
  1.0,
  0.3333333333333333,
  1.0,
  1.0,
  1.0,
  0.0],
 'recall_ls': [0.5,
  0.2222222222222222,
  1.0,
  1.0,
  0.3333333333333333,
  0.3333333333333333,
  0.5,
  0.5,
  0.5,
  0.5,
  1.0,
  0.6,
  0.8,
  0.8,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0],
 'f1

With hybrid search

In [ ]:
hybrid_output=load_object("output/RAG_res_gpt4o/structured_latest_db_prompts_humantextctxv1_hybrid/RAGbatchstra0n1temp0.0_res_dict.pkl")
hybrid_output_eval = calc_eval_metrics(hybrid_output['full output'][0], moalmanac_db_query_subset['prompt'].tolist(), synthetic_query_therapy_pair_dict)
hybrid_output_eval

{'avg_exact_match_acc': 0.85,
 'avg_partial_match_acc': 0.95,
 'avg_precision': 0.8770833333333334,
 'avg_recall': 0.9222222222222222,
 'avg_f1': 0.8840336134453782,
 'avg_specificity': 0.9977475695423778,
 'exact_match_acc': [True,
  False,
  True,
  True,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True],
 'partial_match_acc': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  True,
  True,
  True,
  True],
 'precision_ls': [1.0,
  0.875,
  1.0,
  0.25,
  1.0,
  0.75,
  1.0,
  0.6666666666666666,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'recall_ls': [1.0,
  0.7777777777777778,
  1.0,
  1.0,
  0.6666666666666666,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'f1_ls': [1.0,
  0.823529411764706,
  1.0,
  0.4,
  0.8,
  0.857142

#### Test on full data

In [35]:
moalmanac_db_query.to_csv(f"data/latest_db/moalmanac_core_query__{_VERSION}.csv", index=True)

In [17]:
full_hybrid_output=load_object("output/RAG_res_gpt4o/structured_latest_db_prompts_humantextctxv1_hybrid/RAGstra0n1temp0.0_res_dict.pkl")
full_hybrid_output_eval = calc_eval_metrics(full_hybrid_output['full output'][0], moalmanac_db_query['prompt'].tolist(), synthetic_query_therapy_pair_dict)
full_hybrid_output_eval

{'avg_exact_match_acc': 0.8940809968847352,
 'avg_partial_match_acc': 0.9953271028037384,
 'avg_precision': 0.8873201305444299,
 'avg_recall': 0.9620456905503638,
 'avg_f1': 0.9061313500703285,
 'avg_specificity': 0.9971181791863922,
 'exact_match_acc': [True,
  True,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,